In [1]:
!pip install torchinfo --quiet
!pip install python-Levenshtein --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.5 MB/s eta 0:00:00


In [2]:
import torch
import random
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as f
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import wandb
import torchaudio.transforms as tat
from torchaudio.models.decoder import cuda_ctc_decoder
import Levenshtein

from sklearn.metrics import accuracy_score
import gc

import glob

import zipfile
from tqdm.auto import tqdm
import os
import datetime

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [3]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"rishitsaxena55","key":"0d69850c02f093d6a44b1f2d7ad52c4c"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
!kaggle competitions download -c hw3p2asr-s24 # Download Data

100% 3.74G/3.74G [01:37<00:00, 42.9MB/s]
100% 3.74G/3.74G [01:37<00:00, 41.0MB/s]


In [5]:
!unzip -q hw3p2asr-s24.zip

In [6]:
!ls

11-785-s24-hw3p2  hw3p2asr-s24.zip  sample_data


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
!ls

11-785-s24-hw3p2  gdrive  hw3p2asr-s24.zip  sample_data


In [9]:
CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [10]:
mfcc_dummy = np.load("/content/11-785-s24-hw3p2/test-clean/mfcc/1089-134686-0000.npy")

In [11]:
mfcc_dummy.shape

(1039, 27)

In [12]:
transcript_dummy = np.load("/content/11-785-s24-hw3p2/dev-clean/transcript/1272-128104-0000.npy")

In [13]:
print(type(transcript_dummy[1:-1]))

<class 'numpy.ndarray'>


In [14]:
class AudioDataset(Dataset):
  def __init__(self, root, phonemes):
    self.phonemes = phonemes
    self.mfcc_dir = os.path.join(root, 'mfcc')
    self.transcript_dir = os.path.join(root, 'transcript')

    mfcc_names = sorted(os.listdir(self.mfcc_dir))
    transcript_names = sorted(os.listdir(self.transcript_dir))

    assert len(mfcc_names) == len(transcript_names)

    self.mfccs, self.transcripts = [], []

    for i in range(len(mfcc_names)):
      mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
      mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
      transcript = np.load(os.path.join(self.transcript_dir, transcript_names[i]))

      if transcript[0] == "[SOS]":
        transcript = transcript[1:]
      if transcript[-1] == "[EOS]":
        transcript = transcript[:-1]

      self.mfccs.append(mfcc)
      self.transcripts.append(transcript)

    self.length = len(self.mfccs)

    phoneme_map = {phoneme: i for i, phoneme in enumerate(phonemes)}

    self.transcripts = [[phoneme_map[p] for p in transcript] for transcript in self.transcripts]

  def __len__(self):
    return self.length

  def __getitem__(self, ind):
    mfcc = torch.FloatTensor(self.mfccs[ind])
    transcript = torch.Tensor(self.transcripts[ind])

    return mfcc, transcript

  def collate_fn(self, batch):
    batched_mfccs, batched_transcripts = zip(*batch)

    mfccs_lens = [mfcc.shape[0] for mfcc in batched_mfccs]
    transcripts_lens = [transcript.shape[0] for transcript in batched_transcripts]

    padded_batched_mfccs = pad_sequence(batched_mfccs, batch_first=True)
    padded_batched_transcripts = pad_sequence(batched_transcripts, batch_first=True)

    time_mask = tat.TimeMasking(time_mask_param=100, iid_masks=True, p=0.8)
    time_masked_padded_batched_mfccs = time_mask(torch.Tensor(padded_batched_mfccs))

    freq_mask = tat.FrequencyMasking(freq_mask_param=4, iid_masks=True)
    time_freq_masked_padded_batched_mfccs = freq_mask(torch.Tensor(time_masked_padded_batched_mfccs))

    return time_freq_masked_padded_batched_mfccs, padded_batched_transcripts, torch.IntTensor(mfccs_lens), torch.IntTensor(transcripts_lens)


In [15]:
class AudioDatasetVal(Dataset):
  def __init__(self, root, phonemes):
    self.phonemes = phonemes
    self.mfcc_dir = os.path.join(root, 'mfcc')
    self.transcript_dir = os.path.join(root, 'transcript')

    mfcc_names = sorted(os.listdir(self.mfcc_dir))
    transcript_names = sorted(os.listdir(self.transcript_dir))

    assert len(mfcc_names) == len(transcript_names)

    self.mfccs, self.transcripts = [], []

    for i in range(len(mfcc_names)):
      mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
      mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
      transcript = np.load(os.path.join(self.transcript_dir, transcript_names[i]))

      if transcript[0] == "[SOS]":
        transcript = transcript[1:]
      if transcript[-1] == "[EOS]":
        transcript = transcript[:-1]

      self.mfccs.append(mfcc)
      self.transcripts.append(transcript)

    self.length = len(self.mfccs)

    phoneme_map = {phoneme: i for i, phoneme in enumerate(phonemes)}

    self.transcripts = [[phoneme_map[p] for p in transcript] for transcript in self.transcripts]

  def __len__(self):
    return self.length

  def __getitem__(self, ind):
    mfcc = torch.FloatTensor(self.mfccs[ind])
    transcript = torch.Tensor(self.transcripts[ind])

    return mfcc, transcript

  def collate_fn(self, batch):
    batched_mfccs, batched_transcripts = zip(*batch)

    mfccs_lens = [mfcc.shape[0] for mfcc in batched_mfccs]
    transcripts_lens = [transcript.shape[0] for transcript in batched_transcripts]

    padded_batched_mfccs = pad_sequence(batched_mfccs, batch_first=True)
    padded_batched_transcripts = pad_sequence(batched_transcripts, batch_first=True)

    return padded_batched_mfccs, padded_batched_transcripts, torch.IntTensor(mfccs_lens), torch.IntTensor(transcripts_lens)


In [16]:
class AudioDatasetTest(Dataset):
  def __init__(self, root):
    self.mfcc_dir = os.path.join(root, 'mfcc')

    mfcc_names = sorted(os.listdir(self.mfcc_dir))

    self.mfccs = []

    for i in range(len(mfcc_names)):
      mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
      mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
      self.mfccs.append(mfcc)

    self.length = len(self.mfccs)

  def __len__(self):
    return self.length

  def __getitem__(self, ind):
    mfcc = torch.FloatTensor(self.mfccs[ind])
    return mfcc

  def collate_fn(self, batch):
    batched_mfccs = batch

    mfccs_lens = [mfcc.shape[0] for mfcc in batched_mfccs]

    padded_batched_mfccs = pad_sequence(batched_mfccs, batch_first=True)

    return padded_batched_mfccs, torch.IntTensor(mfccs_lens)


In [117]:
root = "/content/11-785-s24-hw3p2"

config = {
    "beam_width":5,
    "lr": 2e-3,
    "epochs": 50,
    "batch_size": 64,
    "dropout": 0.2,
    "finetune_epochs": 10
}

In [18]:
import gc
gc.collect()

0

In [19]:
train_dir = "/content/11-785-s24-hw3p2/train-clean-100"
val_dir = "/content/11-785-s24-hw3p2/dev-clean"
test_dir = "/content/11-785-s24-hw3p2/test-clean"

In [20]:
train_dataset = AudioDataset(train_dir, PHONEMES)
val_dataset = AudioDatasetVal(val_dir, PHONEMES)
test_dataset = AudioDatasetTest(test_dir)

In [21]:
train_loader = DataLoader(train_dataset, batch_size = config['batch_size'], shuffle=True, collate_fn = train_dataset.collate_fn, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size = config['batch_size'], shuffle=False, collate_fn = val_dataset.collate_fn, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = config['batch_size'], shuffle=False, collate_fn = test_dataset.collate_fn, num_workers=2, pin_memory=True)

In [22]:
print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_dataset.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_dataset.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_dataset.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [120]:
for data in train_loader:
  x, y, lx, ly= data
  print(x.shape, y.shape, lx.shape, ly.shape)
  break

torch.Size([64, 1654, 27]) torch.Size([64, 180]) torch.Size([64]) torch.Size([64])


In [121]:
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
  def forward(self, x):
    return x.transpose(1, 2)

In [122]:
class pBLSTM(torch.nn.Module):
  def __init__(self, input_size, hidden_size):
    super(pBLSTM, self).__init__()

    self.blstm1 = nn.LSTM(input_size*2, hidden_size, batch_first=True, bidirectional=True, dropout=0.2)

  def forward(self, x_packed):
    x_unpacked, lens_unpacked = pad_packed_sequence(x_packed, batch_first=True)

    x_reshaped, x_lens_reshaped = self.trunc_reshape(x_unpacked, lens_unpacked)

    x_packed = pack_padded_sequence(x_reshaped, x_lens_reshaped, enforce_sorted=False, batch_first=True)

    out, _ = self.blstm1(x_packed)

    return out

  def trunc_reshape(self, x, x_lens):
    T = x.shape[1]
    if T % 2 != 0:
      x = x[:, :-1, :]
      x_lens = x_lens - 1

    B, T, F = x.shape

    x = torch.reshape(x, (B, T//2, F*2))
    x_lens = torch.clamp(x_lens // 2, min=1)

    return x, x_lens


In [123]:
class LockedDropout(nn.Module):
    """ LockedDropout applies the same dropout mask to every time step.

    **Thank you** to Sales Force for their initial implementation of :class:`WeightDrop`. Here is
    their `License
    <https://github.com/salesforce/awd-lstm-lm/blob/master/LICENSE>`__.

    Args:
        p (float): Probability of an element in the dropout mask to be zeroed.
    """

    def __init__(self, p=0.5):
        self.p = p
        super().__init__()

    def forward(self, x):
        """
        Args:
            x (:class:`torch.FloatTensor` [sequence length, batch size, rnn hidden size]): Input to
                apply dropout too.
        """
        if not self.training or not self.p:
            return x
        x = x.clone()
        mask = x.new_empty(1, x.size(1), x.size(2), requires_grad=False).bernoulli_(1 - self.p)
        mask = mask.div_(1 - self.p)
        mask = mask.expand_as(x)
        return x * mask


    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'p=' + str(self.p) + ')'

In [124]:
class Unpack(torch.nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x_packed):
    x_unpacked, x_lens = pad_packed_sequence(x_packed, batch_first=True)

    return x_unpacked, x_lens


In [125]:
class Pack(torch.nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x, x_lens):
    x_packed = pack_padded_sequence(x, x_lens, enforce_sorted=False, batch_first=True)

    return x_packed

In [126]:
class Encoder(torch.nn.Module):
  def __init__(self, input_size, encoder_hidden_size):
    super(Encoder, self).__init__()

    self.permute = PermuteBlock()
    self.embedding = nn.Conv1d(input_size, 128, kernel_size=3, padding=1, stride=1)

    self.pBLSTMs = torch.nn.Sequential(
        pBLSTM(128, encoder_hidden_size),
        pBLSTM(2*encoder_hidden_size, encoder_hidden_size),
    )

    self.locked_dropout = LockedDropout()

    self.pack = Pack()
    self.unpack = Unpack()

    self._init_weights()

  def forward(self, x, x_lens):
    x = self.permute(x)
    x = self.embedding(x)
    x = self.permute(x)

    for layer in self.pBLSTMs:
      x = self.pack(x, x_lens)
      x = layer(x)
      x, x_lens = self.unpack(x)
      x = self.permute(x)
      x = self.locked_dropout(x)
      x = self.permute(x)

    encoder_outputs, encoder_lens = (x, x_lens)

    return encoder_outputs, encoder_lens

  def _init_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv1d):
        nn.init.xavier_normal_(m.weight)


In [127]:
class Decoder(torch.nn.Module):
  def __init__(self, embed_size, output_size=41):
    super().__init__()

    self.mlp = nn.Sequential(
        PermuteBlock(), nn.BatchNorm1d(2*embed_size), PermuteBlock(),
        torch.nn.Linear(2*embed_size, 256),
        PermuteBlock(), nn.BatchNorm1d(256), PermuteBlock(),
        torch.nn.GELU(),
        torch.nn.Dropout(p=0.15),
        torch.nn.Linear(256, 256),
        PermuteBlock(), nn.BatchNorm1d(256), PermuteBlock(),
        torch.nn.GELU(),
        torch.nn.Dropout(p=0.15),
        torch.nn.Linear(256, 256),
        PermuteBlock(), nn.BatchNorm1d(256), PermuteBlock(),
        torch.nn.GELU(),
        torch.nn.Dropout(p=0.15),
        torch.nn.Linear(256, 256),
        PermuteBlock(), nn.BatchNorm1d(256), PermuteBlock(),
        torch.nn.GELU(),
        torch.nn.Dropout(p=0.15),
        torch.nn.Linear(256, 128),
        PermuteBlock(), nn.BatchNorm1d(128), PermuteBlock(),
        torch.nn.GELU(),
        torch.nn.Dropout(p=0.15),
        torch.nn.Linear(128, 128),
        PermuteBlock(), nn.BatchNorm1d(128), PermuteBlock(),
        torch.nn.GELU(),
        torch.nn.Linear(128, 64),
        PermuteBlock(), nn.BatchNorm1d(64), PermuteBlock(),
        torch.nn.GELU(),
        torch.nn.Linear(64, 128),
        PermuteBlock(), nn.BatchNorm1d(128), PermuteBlock(),
        torch.nn.GELU(),
        torch.nn.Linear(128, 64),
        PermuteBlock(), nn.BatchNorm1d(64), PermuteBlock(),
        torch.nn.GELU(),
        torch.nn.Linear(64, output_size)
    )

    self.softmax = nn.LogSoftmax(dim=2)

    self._init_weights()

  def forward(self, encoder_out):
    out = self.mlp(encoder_out)
    out = self.softmax(out)

    return out

  def _init_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)


In [128]:
class ASRModel(torch.nn.Module):
  def __init__(self, input_size, embed_size=192, output_size=len(PHONEMES)):
    super().__init__()

    self.encoder = Encoder(input_size, embed_size)
    self.decoder = Decoder(embed_size, output_size)

  def forward(self, x, x_lens):
    encoder_out, encoder_lens = self.encoder(x, x_lens)
    decoder_out = self.decoder(encoder_out)

    return decoder_out, encoder_lens


In [129]:
IN_SIZE = 27
EMBED_SIZE = 192
OUT_SIZE = len(PHONEMES)

In [130]:
model = ASRModel(input_size = IN_SIZE,
                 embed_size = EMBED_SIZE,
                 output_size = OUT_SIZE
).to(device)
print(model)

ASRModel(
  (encoder): Encoder(
    (permute): PermuteBlock()
    (embedding): Conv1d(27, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm1): LSTM(256, 192, batch_first=True, dropout=0.2, bidirectional=True)
      )
      (1): pBLSTM(
        (blstm1): LSTM(768, 192, batch_first=True, dropout=0.2, bidirectional=True)
      )
    )
    (locked_dropout): LockedDropout(p=0.5)
    (pack): Pack()
    (unpack): Unpack()
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): PermuteBlock()
      (1): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PermuteBlock()
      (3): Linear(in_features=384, out_features=256, bias=True)
      (4): PermuteBlock()
      (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): PermuteBlock()
      (7): GELU(approximate='none')
      (8): Dropout(p=0.15, inplace=False)
      (9): Linear(in_features=256, out_fe

In [131]:
summary(model, input_data=(x.to(device), lx))

Layer (type:depth-idx)                   Output Shape              Param #
ASRModel                                 [64, 413, 41]             --
├─Encoder: 1-1                           [64, 413, 384]            --
│    └─PermuteBlock: 2-1                 [64, 27, 1654]            --
│    └─Conv1d: 2-2                       [64, 128, 1654]           10,496
│    └─PermuteBlock: 2-3                 [64, 1654, 128]           --
│    └─Pack: 2-4                         [75825, 128]              --
│    └─Sequential: 2-11                  --                        (recursive)
│    │    └─pBLSTM: 3-1                  [37898, 384]              691,200
│    └─Unpack: 2-6                       [64, 827, 384]            --
│    └─PermuteBlock: 2-7                 [64, 384, 827]            --
│    └─LockedDropout: 2-8                [64, 384, 827]            --
│    └─PermuteBlock: 2-9                 [64, 827, 384]            --
│    └─Pack: 2-10                        [37898, 384]              

In [132]:
print(config)

{'beam_width': 5, 'lr': 0.002, 'epochs': 50, 'batch_size': 64, 'dropout': 0.2, 'finetune_epochs': 10}


In [133]:
criterion = nn.CTCLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'])

decoder = cuda_ctc_decoder(tokens=PHONEMES, nbest=1, beam_size=config['beam_width'])

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

scaler = torch.cuda.amp.GradScaler()

In [134]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):
  output = output.contiguous()
  output_lens = output_lens.to(torch.int32).contiguous()
  results = decoder(output, output_lens)

  pred_strings = []

  for i in range(output_lens.shape[0]):
    pred_strings.append(" ".join([PHONEME_MAP[token] for token in results[i][0].tokens]))

  return pred_strings



In [135]:
def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP=LABELS):
  dist = 0
  batch_size = label.shape[0]

  pred_strings = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

  for i in range(batch_size):
    pred_string = pred_strings[i]
    label_string = " ".join([PHONEME_MAP[int(token)] for token in label[i][:int(label_lens[i])].tolist()])

    dist += Levenshtein.distance(pred_string, label_string)

  dist /= batch_size

  return dist


In [143]:
model.eval()

for i, data in enumerate(val_loader, 0):
  x, y, lx, ly = data
  x, y = x.to(device), y.to(device)
  #lx, ly = lx.to(device), ly.to(device)

  h, lh = model(x, lx)
  print(h.shape)
  h = torch.permute(h, (1, 0, 2))
  print(h.shape, y.shape)

  loss = criterion(h, y, lh, ly)
  print(loss)

  print(calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh.to(device), ly, decoder, LABELS))

  del x, y, lx, ly, h, lh, loss
  torch.cuda.empty_cache()

  break

torch.Size([64, 734, 41])
torch.Size([734, 64, 41]) torch.Size([64, 265])
tensor(0.6430, device='cuda:0', grad_fn=<MeanBackward0>)
15.6875


In [40]:
import wandb
wandb.login(key="39e9c89279f6d046c7bae725e099c70ddf0fd98f")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rishitsaxena55 (rishitsaxena55-indian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
run = wandb.init(
    name = "early-submission", ## Wandb creates random run names if you skip this field
    #reinit = True, ### Allows reinitalizing runs when you re-run this cell
    id = "rbborunl", ### Insert specific run id here if you want to resume a previous run
    resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

lr,0.01
train_loss,3.00393
valid_dist,125.91095
valid_loss,2.8879


In [138]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh.to(device), ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

In [139]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_dist', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [140]:
model, optimizer, scheduler, epoch, metric = load_model("/content/gdrive/MyDrive/best_model.pth", model= model, metric= "valid_dist", optimizer= optimizer, scheduler= scheduler)

In [141]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = epoch
start = last_epoch_completed + 1
end = config["finetune_epochs"]
best_lev_dist = metric # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = "/content/gdrive/MyDrive" #TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = "/content/gdrive/MyDrive" #TODO set best model path

In [44]:
config['finetune_epochs']

10

In [45]:
config['beam_width']

10

In [46]:
config['lr']

0.002

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(start, start + config['finetune_epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, start +config['finetune_epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_loss              = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, LABELS)
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    # save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, os.path.join(epoch_model_path, f"model_epoch_{epoch+1}.pth"))
    # wandb.save(os.path.join(epoch_model_path, f"model_epoch_{epoch+1}.pth"))
    # print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, os.path.join(best_model_path, f"best_model_epoch_{epoch+1}.pth"))
        wandb.save(os.path.join(best_model_path, f"best_model_epoch_{epoch+1}.pth"))
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 53/62


	Train Loss 1.2752	 Learning Rate 0.0100000
	Val Dist 30.2055%	 Val Loss 1.0833

Epoch: 54/62


	Train Loss 1.6721	 Learning Rate 0.0100000
	Val Dist 41.1797%	 Val Loss 1.3862

Epoch: 55/62


	Train Loss 2.5240	 Learning Rate 0.0100000
	Val Dist 126.3271%	 Val Loss 2.9478

Epoch: 56/62


	Train Loss 3.0039	 Learning Rate 0.0100000
	Val Dist 125.9109%	 Val Loss 2.8879

Epoch: 57/62


Train:  83%|████████▎ | 371/446 [02:09<00:25,  2.98it/s, loss=3.0923, lr=0.010000]

KeyboardInterrupt: 

In [ ]:
def test(test_decoder, test_beam_width, model, test_loader):

  results = []
  model.eval()
  print("Testing")
  for data in tqdm(test_loader):

    x, lx = data
    x = x.to(device)

    with torch.no_grad():
      h, lh = model(x, lx)

    prediction_string = decode_prediction(h, lh.to(device), test_decoder, LABELS)
    results.extend(prediction_string)

    del x, lx, h, lh
    torch.cuda.empty_cache()



In [144]:
TEST_BEAM_WIDTH = 10

test_decoder = cuda_ctc_decoder(tokens= PHONEMES, nbest= 1, beam_size= TEST_BEAM_WIDTH)
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

  x, lx = data
  x = x.to(device)

  with torch.no_grad():
    h, lh = model(x, lx)

  prediction_string = decode_prediction(h, lh.to(device), test_decoder, LABELS)
  results.extend(prediction_string)

  del x, lx, h, lh
  torch.cuda.empty_cache()

Testing


100%|██████████| 41/41 [00:06<00:00,  6.69it/s]


In [145]:
results = [s.replace(" ", "") for s in results]

In [146]:
print(results)

['-hihoptDErwUdbistufcrdInR-tRnApsAndk@rAtsAndbruzdpAtetozAndf@tmAtAnpisAztubiletAldWtAndTIkpEpRdflWRf@kAndscs-', '-stAfAdIntuyu-hIzbElikWntAldhIm-', '-@ftRRlinYtfclDAyEnol@mpswUdlYthWphirAndDErDAskwalAdkwcrtRAvDAbrafAlz-', '-WbRtiAndigUdInhrmYnd-', '-nAmbR-dEn-frESnEliAzwetIGanyu-kUdnathAzbAnd-', '-DAmyuzAkemnIrAndhirAkcldDAwRdz-DAwRdzAvSElizfrYgmAntApanDAmunwandRIGkAmp@nyAnlAs-pelfcrwIrinAs-', '-DAdolYtufElmcrfentliApanDApejwEranAnADRrikweSAnbIg@ntuAnfoldItsElfsloli-AndtusprEdAvbrcdItswYdAnIGtel-', '-AkoldlusAdIndIfRAnsRendInhIzso-', '-DAkeasAndwIChIzardRIkstIGgwIStItsElfwazAkuldIndIfRAnt-nalAjAvhImsElf-', '-@tmost-bY@nclmzgIvAntuAbEgRhuzblEsIGhiflEdfrAm-himYthopwIrlituwInfcrhImsElf-sAmmEZRAv@kCAwAlgres-', '-wEldWInAsYdIklIryuh@vAhEdAndsoh@zmYstIk-', '-ans@tRdimcrnIGzwEnDAsod@lAthimEtInDAC@pAlturIsYdDAlItAlcfAs-hIzpleswazAkwISAndnilIGdEsk@tDArYtAvDAcltRfrAmwIChilEdhIzwIGAvbOzTruDArIspansAz-', '-hRYzsimdturAgardhImwIDmYldpIti-hRholIGAs-AstrenjlYtgloIGfetliApanhRfrelflES-dIdnathimIliet

In [147]:
data_dir = f"{root}/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission4.csv', index=False)